In [1]:
from __future__ import division
import ROOT
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Wedge, RegularPolygon
from matplotlib.collections import PatchCollection

from AnalysisEvent import AnalysisEvent
import math


In [2]:
ROOT.gSystem.Load("/home/felipe/MG5_aMC_v2_5_5/Delphes/libDelphes")

0

In [3]:
events = AnalysisEvent("pp_ttbar_CMS_goodbtag.root")

TClass::TClass:0: RuntimeWarning: no dictionary for class ROOT::TIOFeatures is available


In [4]:
events.addCollection("jets","Jet")
events.addCollection("genjets","GenJet")
events.addCollection("electrons","Electron")
events.addCollection("muons","Muon")
events.addCollection("particles","Particle")
events.addCollection("towers","Tower")
events.addCollection("tracks","Track")
events.addCollection("eflowtracks","EFlowTrack")
events.addCollection("eflowphotons","EFlowPhoton")
events.addCollection("eflowneutralhads","EFlowNeutralHadron")

In [7]:
class DetectorImage:
    shapescale = 0.3
    
    deta = 0.02
    dphi = 0.02
    
    etamin = -5.0
    etamax = 5.0

    phimin = -math.pi
    phimax = math.pi

    n_bins_eta = int( (etamax-etamin)/deta)
    n_bins_phi = int( (phimax-phimin)/dphi)
    
    def get_image_from_particles(self,collection,idx,xsize,ysize,dpiS,fc='k'):
        
        img = np.zeros( (self.n_bins_eta, self.n_bins_phi) , np.int16 )

        def find_bin(eta,phi):
            
            #print "Check ",phi," ",self.phimin," ",self.dphi
            etabin = int((eta-self.etamin)/self.deta)
            phibin = int((phi-self.phimin)/self.dphi)

            return (etabin,phibin)
            
        for track in collection:

            (etabin, phibin) = find_bin(track.Eta,track.Phi)

            #print (track.Eta,track.Phi)," ",(etabin, phibin)

            if hasattr(track,"PT"):
                try:
                    img[etabin,phibin] = img[etabin,phibin] + track.PT
                except IndexError:
                    img[0,0] = -999
            else:
                try:
                    img[etabin,phibin] = img[etabin,phibin] + track.ET
                except IndexError:
                    img[0,0] = -999

    def get_image_from_event(self,event,idx,xsize,ysize,dpiS,fc='k'):
        rgb_color = []
        line_with = (xsize/dpiS)*0.1
        if fc is not 'k':
            rgb_color = ['r','g','b']


        fig, ax = plt.subplots(subplot_kw=dict(xlim=(self.etamin,self.etamax),ylim=(self.phimin,self.phimax)),
                              figsize=(xsize/dpiS,ysize/dpiS),dpi=dpiS,facecolor=fc)        
        patches = []
        ax.axis('off')
        ax.set_facecolor(fc)

        for el in event.EFlowTrack:
            coords = ( el.Eta, el.Phi )
            
            #print "Circle at ",coords, math.log(el.PT), " pT ",el.PT

            if fc is 'k':
                circle = Circle(coords, self.shapescale*math.log(el.PT),
                            fill=False,lw=line_with, color='w')
            else:
                pass
            
            if fc is 'w':
                circle = Circle(coords, self.shapescale*math.log(el.PT),
                            fill=False,lw=line_with, color=rgb_color[0])
                
            ax.add_artist(circle)
            
        for el in event.EFlowPhoton:
            coords = ( el.Eta, el.Phi )
            
            #print "Triangle at ",coords, math.log(el.ET), " ET ",el.ET
            if fc is 'k':
                square = RegularPolygon(coords, 4, self.shapescale*math.log(el.ET),
                                    fill=False,lw=line_with, color='w')
            else:
                pass

            if fc is 'w':
                square = RegularPolygon(coords, 4, self.shapescale*math.log(el.ET),
                                    fill=False,lw=line_with, color=rgb_color[1])
            
                
            ax.add_artist(square)
            
        for el in event.EFlowNeutralHadron:
            coords = ( el.Eta, el.Phi )
            
            #print "Pentagon at ",coords, math.log(el.ET), " ET ",el.ET
            if fc is 'k':
                hexagon = RegularPolygon(coords, 6, self.shapescale*math.log(el.ET),
                                     fill=False,lw=line_with, color='w')
            else:
                pass

            if fc is 'w':
                hexagon = RegularPolygon(coords, 6, self.shapescale*math.log(el.ET),
                                     fill=False,lw=line_with, color=rgb_color[2])
                    
            ax.add_artist(hexagon)
            
                    
#        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
#        fig.savefig('ww_plots/plotcolors_{}.png'.format(idx),bbox_inches = extent,facecolor=ax.get_facecolor())        

        fig.canvas.draw()
        
        w,h = fig.canvas.get_width_height()
        data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')        
        data.shape = ( w, h, 3 )
        if fc is 'k':
            np.save('ttbar_plots/data_bw_{}_w_{}_h_{}'.format(idx,w,h), data)
        else:
            pass

        if fc is 'w':
            np.save('ttbar_plots/data_color_{}_w_{}_h_{}'.format(idx,w,h), data)

        plt.close()                    

In [9]:
cnt = 0
for event in events:
    cnt +=1
    if (cnt==10000):
        break

    if (cnt%100==0):
        print "Processing event ",cnt
    
    dimage = DetectorImage()

    img_photons = dimage.get_image_from_particles(event.EFlowPhoton,cnt,224,224,224,fc='k')
    img_tracks = dimage.get_image_from_particles(event.EFlowTrack,cnt,224,224,224,fc='k')
    img_neutrals = dimage.get_image_from_particles(event.EFlowNeutralHadron,cnt,224,224,224,fc='k')
    
    dimage =  dimage.get_image_from_event(event,cnt,224,224,224,fc='k')


Processing event  100
Processing event  200
Processing event  300
Processing event  400
Processing event  500
Processing event  600
Processing event  700
Processing event  800
Processing event  900
Processing event  1000
Processing event  1100
Processing event  1200
Processing event  1300
Processing event  1400
Processing event  1500
Processing event  1600
Processing event  1700
Processing event  1800
Processing event  1900
Processing event  2000
Processing event  2100
Processing event  2200
Processing event  2300
Processing event  2400
Processing event  2500
Processing event  2600
Processing event  2700
Processing event  2800
Processing event  2900
Processing event  3000
Processing event  3100
Processing event  3200
Processing event  3300
Processing event  3400
Processing event  3500
Processing event  3600
Processing event  3700
Processing event  3800
Processing event  3900
Processing event  4000
Processing event  4100
Processing event  4200
Processing event  4300
Processing event  44